In [1]:
import pandas as pd
import toml
from neo4j import GraphDatabase
from tqdm import tqdm
# import logging # introduce logging later when exporting this to .py or deploying

# # Set up logging
# logging.basicConfig(level=logging.INFO)
# logger = logging.getLogger(__name__)



## Phase 1: Load and Inspect CSVs

In [2]:

# CSV file paths
results_csv_path = "data/results.csv"
goalscorers_csv_path = "data/goalscorers.csv"
shootouts_csv_path = "data/shootouts.csv"

# logger.info("Loading data...")
results_df = pd.read_csv(results_csv_path, parse_dates=["date"])
goalscorers_df = pd.read_csv(goalscorers_csv_path, parse_dates=["date"])
shootouts_df = pd.read_csv(shootouts_csv_path, parse_dates=["date"])


results_df.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False


## Phase 2: Connect to AuraDB with Neo4j Python Driver

In [3]:

# Read secrets
secrets = toml.load(".streamlit/secrets.toml")

NEO4J_URI = secrets["NEO4J_URI"]
NEO4J_USER = secrets["NEO4J_USER"]
NEO4J_PASSWORD = secrets["NEO4J_PASSWORD"]

# Connect to Neo4j
driver = GraphDatabase.driver(uri=NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
print("Connected to AuraDB")

Connected to AuraDB


In [4]:
# Testing
with driver.session() as session:
    result = session.run("RETURN 'Neo4j is ready!' AS message")
    print(result.single()["message"])


Neo4j is ready!


In [5]:
print(results_df.shape)
print(goalscorers_df.shape)
print(shootouts_df.shape)

(48366, 9)
(44447, 8)
(650, 5)


## Phase 3: Design Your Graph Schema (Nodes + Relationships)

## 🧱 Graph Schema:

#### 🟦 Nodes

* `(:Team {name})`
* `(:Player {name})`
* `(:Match {date, home_score, away_score, tournament, city, country, neutral})`

#### 🔗 Relationships

* `(Team)-[:PLAYED_HOME]->(Match)`
* `(Team)-[:PLAYED_AWAY]->(Match)`
* `(Player)-[:SCORED_IN {minute, own_goal, penalty}]->(Match)`
* `(Player)-[:SCORED_FOR]->(Team)`
* `(Team)-[:WON_SHOOTOUT]->(Match)`
* `(Team)-[:FIRST_SHOOTER_IN]->(Match)`

## Phase 4: Ingest results.csv (Teams + Matches + Relationships)


In [6]:
# Step 1: Create Unique :Team Nodes

# Get all team names from both home and away columns
home_teams = results_df["home_team"].unique()
away_teams = results_df["away_team"].unique()
all_teams = set(home_teams).union(set(away_teams))

total_teams = len(all_teams)
print(f"Total number of unique teams: {total_teams}")

Total number of unique teams: 332


In [7]:
# define a Cypher query method to merge (create if not exists) the team nodes:

def create_teams(driver, team_names):
    """
    Create or ensure the existence of unique Team nodes in the Neo4j graph.

    This function receives a list of team names and uses a Cypher query
    with UNWIND and MERGE to insert each name as a :Team node in the database.
    Duplicate nodes are avoided automatically via MERGE.

    Args:
        driver (neo4j.GraphDatabase.driver): The active Neo4j driver instance.
        team_names (Iterable[str]): A list or set of unique team names.

    Returns:
        None. Prints a success message upon completion.
    """
    query = "UNWIND $teams AS team_name MERGE (:Team {name: team_name})"
    with driver.session() as session:
        session.run(query, teams=list(team_names))
    print(f"✅ Created {len(team_names)} Team nodes")


In [8]:
create_teams(driver=driver, team_names=all_teams)

✅ Created 332 Team nodes


In [9]:
def chunk_list(data, chunk_size):
    """
    Yield successive chunks from a list.

    This function splits a list into smaller lists (batches) of a specified size,
    which is useful for batch operations like database writes or API calls.

    Args:
        data (List[Any]): The full list to be chunked.
        chunk_size (int): The number of items per chunk.

    Yields:
        List[Any]: A sublist (chunk) of the original list.
    """
    for i in range(0, len(data), chunk_size):
        yield data[i:i + chunk_size]


In [10]:
def prepare_match_rows(df):
    """
    Convert a match DataFrame into a list of dictionary records for Cypher ingestion.

    Each row is converted into a Python dictionary with cleaned and typed values,
    including converting dates to string format for Neo4j compatibility.

    Args:
        df (pd.DataFrame): The input DataFrame from results.csv with parsed dates.

    Returns:
        List[Dict[str, Any]]: A list of match dictionaries ready for batch Cypher upload.
    """
    match_rows = []
    for _, row in df.iterrows():
        match = {
            "date": row["date"].strftime("%Y-%m-%d"),
            "home_team": row["home_team"],
            "away_team": row["away_team"],
            "home_score": int(row["home_score"]),
            "away_score": int(row["away_score"]),
            "tournament": row["tournament"],
            "city": row["city"],
            "country": row["country"],
            "neutral": bool(row["neutral"])
        }
        match_rows.append(match)
    return match_rows


In [11]:
# Step 2: Create :Match Nodes + Connect Teams
# Let’s now go row-by-row in results_df and create matches and relationships.


def create_matches(driver, match_rows, batch_size=500):
    """
    Batch-ingest Match nodes and their team relationships into Neo4j.

    For each match in the batch, this function:
    - Creates home and away Team nodes (if they don't exist)
    - Creates a Match node with key and detailed properties
    - Connects each team to the match using PLAYED_HOME / PLAYED_AWAY

    Args:
        driver: Neo4j GraphDatabase driver
        match_rows: list of dicts containing match data
        batch_size: number of matches per Cypher query
    """
    query = """
    UNWIND $batch AS row
    MERGE (home:Team {name: row.home_team})
    MERGE (away:Team {name: row.away_team})
    MERGE (m:Match {
        date: row.date,
        home_team: row.home_team,
        away_team: row.away_team
    })
    SET m.home_score = row.home_score,
        m.away_score = row.away_score,
        m.tournament = row.tournament,
        m.city = row.city,
        m.country = row.country,
        m.neutral = row.neutral
    MERGE (home)-[:PLAYED_HOME]->(m)
    MERGE (away)-[:PLAYED_AWAY]->(m)
    """
    with driver.session() as session:
        for batch in tqdm(chunk_list(match_rows, batch_size), total=len(match_rows)//batch_size + 1):
            session.run(query, batch=batch)
    print(f"✅ Ingested {len(match_rows)} Matches using batch size {batch_size}")

In [12]:
match_rows = prepare_match_rows(results_df)   # Step 1
create_matches(driver, match_rows)            # Step 2


100%|██████████| 97/97 [13:13<00:00,  8.18s/it]

✅ Ingested 48366 Matches using batch size 500


## Phase 5: Ingest goalscorers.csv → create Player nodes + SCORED_IN + SCORED_FOR


In [21]:
def prepare_goal_rows(df):
    """
    Prepare a clean, safe list of goal dicts from the goalscorers DataFrame.

    Filters out rows with missing critical fields (scorer, team, match identifiers).
    Safely handles optional fields with default values.

    Args:
        df (pd.DataFrame): The goalscorers DataFrame with parsed dates.

    Returns:
        List[Dict[str, Any]]: Cleaned list of goal records.
    """
    goal_rows = []

    for _, row in df.iterrows():
        # Validate required fields
        if pd.isna(row["scorer"]) or pd.isna(row["team"]) or pd.isna(row["home_team"]) or pd.isna(row["away_team"]):
            continue  # Skip rows with missing critical fields

        match = {
            "date": row["date"].strftime("%Y-%m-%d") if pd.notna(row["date"]) else None,
            "home_team": row["home_team"],
            "away_team": row["away_team"],
            "team": row["team"],
            "scorer": row["scorer"],
            "minute": int(row["minute"]) if pd.notna(row["minute"]) else -1,
            "own_goal": bool(row["own_goal"]) if pd.notna(row["own_goal"]) else False,
            "penalty": bool(row["penalty"]) if pd.notna(row["penalty"]) else False
        }

        # Avoid adding rows with missing match keys
        if not match["date"] or not match["home_team"] or not match["away_team"]:
            continue

        goal_rows.append(match)

    return goal_rows


In [19]:

def create_goals(driver, goal_rows, batch_size=500):
    """
    Batch-ingest Player nodes and connect them to Match and Team nodes.

    For each goal:
    - Creates a Player node (if not exists)
    - Finds the corresponding Match node (by date + teams)
    - Connects Player to Match with SCORED_IN (adds minute, own_goal, penalty)
    - Connects Player to Team with SCORED_FOR

    Args:
        driver: Neo4j driver instance
        goal_rows: List of goal dictionaries (from prepare_goal_rows)
        batch_size: Number of goals per transaction batch

    Returns:
        None. Prints success summary after ingestion.
    """
    query = """
    UNWIND $batch AS row
    MERGE (p:Player {name: row.scorer})
    MERGE (t:Team {name: row.team})
    WITH p, t, row
    MATCH (m:Match {
        date: row.date,
        home_team: row.home_team,
        away_team: row.away_team
    })
    MERGE (p)-[:SCORED_IN {
        minute: row.minute,
        own_goal: row.own_goal,
        penalty: row.penalty
    }]->(m)
    MERGE (p)-[:SCORED_FOR]->(t)
    """


    with driver.session() as session:
        for batch in tqdm(chunk_list(goal_rows, batch_size), total=len(goal_rows)//batch_size + 1):
            session.run(query, batch=batch)
    print(f"✅ Ingested {len(goal_rows)} goals using batch size {batch_size}")


In [22]:
goal_rows = prepare_goal_rows(goalscorers_df)
create_goals(driver, goal_rows)


100%|██████████| 89/89 [27:22<00:00, 18.45s/it]

✅ Ingested 44399 goals using batch size 500


## Phase 6: Ingest shootouts.csv → model penalty shootout winners.

In [23]:
shootouts_df.columns

Index(['date', 'home_team', 'away_team', 'winner', 'first_shooter'], dtype='object')

In [24]:
def prepare_shootout_rows(df):
    """
    Prepare a list of clean dictionaries from the shootouts DataFrame.

    Each row captures the shootout outcome for a given match:
    - The team that won the shootout
    - The team that kicked first

    Args:
        df (pd.DataFrame): The shootouts DataFrame with parsed dates.

    Returns:
        List[Dict[str, Any]]: Cleaned and filtered shootout records.
    """
    rows = []
    for _, row in df.iterrows():
        # Skip if any critical field is missing
        if pd.isna(row["date"]) or pd.isna(row["home_team"]) or pd.isna(row["away_team"]):
            continue

        shootout = {
            "date": row["date"].strftime("%Y-%m-%d"),
            "home_team": row["home_team"],
            "away_team": row["away_team"],
            "winner": row["winner"] if pd.notna(row["winner"]) else None,
            "first_shooter": row["first_shooter"] if pd.notna(row["first_shooter"]) else None
        }

        rows.append(shootout)
    return rows


In [27]:

def create_shootouts(driver, shootout_rows, batch_size=500):
    """
    Add shootout relationships to existing matches in the graph.

    For each row:
    - Finds the correct Match node (date, home, away)
    - Connects the winning team with a :WON_SHOOTOUT relationship
    - Connects the first-kicking team with :FIRST_SHOOTER_IN

    Args:
        driver: Neo4j driver instance
        shootout_rows: List of cleaned shootout dicts
        batch_size: Max rows per Cypher batch

    Returns:
        None
    """
    query = """
    UNWIND $batch AS row
    MATCH (m:Match {
        date: row.date,
        home_team: row.home_team,
        away_team: row.away_team
    })

    OPTIONAL MATCH (winner:Team {name: row.winner})
    FOREACH (_ IN CASE WHEN winner IS NOT NULL THEN [1] ELSE [] END |
        MERGE (winner)-[:WON_SHOOTOUT]->(m)
    )

    WITH m, row

    OPTIONAL MATCH (first:Team {name: row.first_shooter})
    FOREACH (_ IN CASE WHEN first IS NOT NULL THEN [1] ELSE [] END |
        MERGE (first)-[:FIRST_SHOOTER_IN]->(m)
    )
    """


    with driver.session() as session:
        for batch in tqdm(chunk_list(shootout_rows, batch_size), total=len(shootout_rows)//batch_size + 1):
            session.run(query, batch=batch)
    print(f"✅ Ingested {len(shootout_rows)} shootout records using batch size {batch_size}")


In [28]:
shootout_rows = prepare_shootout_rows(shootouts_df)
create_shootouts(driver, shootout_rows)


100%|██████████| 2/2 [00:22<00:00, 11.32s/it]

✅ Ingested 650 shootout records using batch size 500
